# ParseTickers Module

## Load get_wikitable_from_url

In [1]:
import re
import os
import bs4 as bs
import pandas as pd
import requests
from datetime import date
from datetime import datetime

In [2]:
wiki_metadata = [{'market':'IBEX35','url':'https://es.wikipedia.org/wiki/IBEX_35','pos_table':{'ticker':0, 'company':1,'sector':4,'entry_date':3,'ISIN':5}},
                {'market':'DAX30','url':'https://de.wikipedia.org/wiki/DAX','pos_table':{'ticker':1, 'company':0,'sector':2,'entry_date':5}},
                {'market':'CAC40','url':'https://es.wikipedia.org/wiki/CAC_40','pos_table':{'ticker':2,'company':0, 'sector':1}},
                {'market':'SP500','url':'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies','pos_table':{'ticker':0, 'company':1,'sector':3,'sub_industry':4,'entry_date':6,'CIK':7}}
                ]

In [3]:
wiki_metadata[0]

{'market': 'IBEX35',
 'url': 'https://es.wikipedia.org/wiki/IBEX_35',
 'pos_table': {'ticker': 0,
  'company': 1,
  'sector': 4,
  'entry_date': 3,
  'ISIN': 5}}

In [4]:
def get_wikitable_from_url(wiki_metadata) -> dict:
    resp = requests.get(wiki_metadata['url'])
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    wikitable_data = []
    for row in table.findAll('tr')[1:]:
        wikitable_dict = {}
        for key in wiki_metadata['pos_table']:
            data = row.findAll('td')[wiki_metadata['pos_table'][key]].text
            wikitable_dict[key] = data.strip()
        wikitable_dict['market'] = wiki_metadata['market']
        wikitable_dict['active_type'] = 'stock'
        wikitable_data.append(wikitable_dict)
    wikitable_data.append({'ticker':wiki_metadata['market'] ,'market':wiki_metadata['market'], 'active_type':'ETF'})
    return wikitable_data

In [5]:
raw_ticker_list = []
print('Start raw ticker data extraction from Wikipedia ')
for i,metadata in enumerate(wiki_metadata):
    print('Getting data from market:{}, site:{}'.format(metadata['market'], metadata['url']))
    wikitable_data = get_wikitable_from_url(metadata)
    raw_ticker_list += wikitable_data
print('Extraction completed')

Start raw ticker data extraction from Wikipedia 
Getting data from market:IBEX35, site:https://es.wikipedia.org/wiki/IBEX_35
Getting data from market:DAX30, site:https://de.wikipedia.org/wiki/DAX
Getting data from market:CAC40, site:https://es.wikipedia.org/wiki/CAC_40
Getting data from market:SP500, site:http://en.wikipedia.org/wiki/List_of_S%26P_500_companies
Extraction completed


In [6]:
raw_ticker_list

[{'ticker': 'ANA',
  'company': 'Acciona',
  'sector': 'Construcción',
  'entry_date': '2015',
  'ISIN': 'ES0125220311',
  'market': 'IBEX35',
  'active_type': 'stock'},
 {'ticker': 'ACX',
  'company': 'Acerinox',
  'sector': 'Mineral, metales y transformación',
  'entry_date': '2015',
  'ISIN': 'ES0132105018',
  'market': 'IBEX35',
  'active_type': 'stock'},
 {'ticker': 'ACS',
  'company': 'Grupo ACS',
  'sector': 'Construcción',
  'entry_date': '1998',
  'ISIN': 'ES0167050915',
  'market': 'IBEX35',
  'active_type': 'stock'},
 {'ticker': 'AENA',
  'company': 'Aena',
  'sector': 'Transporte y distribución',
  'entry_date': '2015',
  'ISIN': 'ES0105046009',
  'market': 'IBEX35',
  'active_type': 'stock'},
 {'ticker': 'ALM',
  'company': 'Almirall',
  'sector': 'Productos farmacéuticos y biotecnología',
  'entry_date': '2020',
  'ISIN': 'ES0157097017',
  'market': 'IBEX35',
  'active_type': 'stock'},
 {'ticker': 'AMS',
  'company': 'Amadeus IT Group',
  'sector': 'Electrónica y software

## Parse_tickers minimum code


- BaseParser(): 
- WikiToYahooFinanceParser()
- get_parsed_tickers(): método de la clase TickerManager_Wiki. Es la api para parse_Tickers.


In [7]:
class BaseParser():
    
    def __init__(self, source, target):
        self.__source = source
        self.__target = target
        self.__exceptions ={('Wiki','YahooFinance'):{'IBEX35': '^IBEX','CAC40': '^FCHI','DAX30': '^GDAXI', 'SP500':'^GSPC'},
                            ('Wiki', 'Ticker_Key'):{'IBEX35': '^IBEX.TT','CAC40': '^FCHI.TT','DAX30': '^GDAXI.TT', 'SP500':'^GSPC.TT'},
                            ('Wiki','AlphaVantage'):{}}
        
    def get_exceptions(self):
        return self.__exceptions
    
    def set_source(self, source):
        print("Set ticker source in parser")
        self.__source = source
    
    def get_source(self):
        return self.__source
    
    def set_target(self, target):
        print("Set ticker target in parser")
        self.__target = target
    
    def get_target(self):
        return self.__target
       
    def parse(self, name, prefix, suffix, excedent_string):
        key_tuple = (self.__source,self.__target) # Tuple key to select Source-Target exceptions
        
        exception_dict = self.__exceptions.get(key_tuple, {}) #FIX inform not properly loaded. Mirar longitud del dict.

        if name in exception_dict:
            return exception_dict.get(name, name) 
        
        name = name.replace(excedent_string,'')
        return prefix + name + suffix
    
    def parse_markets(self, raw_ticker, parser_keys): # Feeds tickers for Yahoo finance downloader based on market. Uses parse from BaseParser
        try:
            return self.parse(raw_ticker['ticker'], 
                     parser_keys[('market', raw_ticker['market'])]['prefix'],
                     parser_keys[('market', raw_ticker['market'])]['suffix'],
                     parser_keys[('market', raw_ticker['market'])]['excedent_string'])
        except:
            return raw_ticker['ticker']

In [8]:
class WikiToYahooFinance_Parser(BaseParser):
    
    def __init__(self):
        super().__init__(source='Wiki', target='YahooFinance')
        
        self.__parser_keys_feed = {('market','IBEX35'):{'prefix':'','suffix':'.MC','excedent_string':''},
                              ('market','CAC40'):{'prefix':'','suffix':'.PA','excedent_string':'Euronext: '},
                              ('market','DAX30'):{'prefix':'','suffix':'.DE', 'excedent_string':''},
                              ('market','SP500'):{'prefix':'','suffix':'', 'excedent_string':''}}
        
        self.__parser_keys_tickerKey = {('market','IBEX35'):{'prefix':'','suffix':'.MC.TT','excedent_string':''},
                                       ('market','CAC40') :{'prefix':'','suffix':'.PA.TT','excedent_string':'Euronext: '},
               ('market','DAX30') :{'prefix':'','suffix':'.DE.TT', 'excedent_string':''},
               ('market','SP500') :{'prefix':'','suffix':'', 'excedent_string':''}}
        
        
    def feeder_ticker(self,feed_ticker): # Appends feeds to original dictionary, under 'feeds' label. Uses parse_market.
        feed_ticker['feeds'] = {'name':self.get_target(),'ticker':self.parse_markets(feed_ticker, self.__parser_keys_feed)} #Feeds
        return feed_ticker

In [9]:
class WikiToTickerKey_Parser(BaseParser):
    
    def __init__(self):
        super().__init__(source='Wiki', target='YahooFinance')
        
        self.__parser_keys_tickerKey = {('market','IBEX35'):{'prefix':'','suffix':'.MC.TT','excedent_string':''},
                                       ('market','CAC40') :{'prefix':'','suffix':'.PA.TT','excedent_string':'Euronext: '},
               ('market','DAX30') :{'prefix':'','suffix':'.DE.TT', 'excedent_string':''},
               ('market','SP500') :{'prefix':'','suffix':'', 'excedent_string':''}}
        
    def feeder_ticker(self,feed_ticker): # Appends feeds to original dictionary, under 'feeds' label. Uses parse_market.
        feed_ticker['tickerKey'] = self.parse_markets(feed_ticker, self.__parser_keys_tickerKey)
        return feed_ticker

In [12]:
def get_parsed_tickers(): # Función con la que hay que engancahr
        # Actualiza los par-value del diccionario: "tickerYahoofinance":"value", "tickerAlphaVantage":"value"
        print('Activation: get_parsed_tickers')
        my_parser_W2YF = WikiToYahooFinance_Parser()
        my_parser_W2TK = WikiToTickerKey_Parser()
        feeder_ticker_list = []
        for raw_ticker in raw_ticker_list:
            feeder_ticker = my_parser_W2TK.feeder_ticker(raw_ticker)
            feeder_ticker = my_parser_W2YF.feeder_ticker(feeder_ticker)
            feeder_ticker_list.append(feeder_ticker)
        print('All tickers fed')
        return feeder_ticker_list

In [13]:
get_parsed_tickers()

Activation: get_parsed_tickers
All tickers fed


[{'ticker': 'ANA',
  'company': 'Acciona',
  'sector': 'Construcción',
  'entry_date': '2015',
  'ISIN': 'ES0125220311',
  'market': 'IBEX35',
  'active_type': 'stock',
  'feeds': {'name': 'YahooFinance', 'ticker': 'ANA.MC'},
  'tickerKey': 'ANA.MC.TT'},
 {'ticker': 'ACX',
  'company': 'Acerinox',
  'sector': 'Mineral, metales y transformación',
  'entry_date': '2015',
  'ISIN': 'ES0132105018',
  'market': 'IBEX35',
  'active_type': 'stock',
  'feeds': {'name': 'YahooFinance', 'ticker': 'ACX.MC'},
  'tickerKey': 'ACX.MC.TT'},
 {'ticker': 'ACS',
  'company': 'Grupo ACS',
  'sector': 'Construcción',
  'entry_date': '1998',
  'ISIN': 'ES0167050915',
  'market': 'IBEX35',
  'active_type': 'stock',
  'feeds': {'name': 'YahooFinance', 'ticker': 'ACS.MC'},
  'tickerKey': 'ACS.MC.TT'},
 {'ticker': 'AENA',
  'company': 'Aena',
  'sector': 'Transporte y distribución',
  'entry_date': '2015',
  'ISIN': 'ES0105046009',
  'market': 'IBEX35',
  'active_type': 'stock',
  'feeds': {'name': 'YahooFinan

### Parse_tickers: Development layer

In [ ]:
def parse(name, prefix, suffix, excedent_string):
        source = 'Wiki'
        target = 'Target'
        exceptions ={('Wiki','YahooFinance'):{'IBEX35': '^IBEX','CAC40': '^FCHI','DAX30': '^GDAXI', 'SP500':'^GSPC'},
                            ('Wiki', 'Ticker_Key'):{'IBEX35': '^IBEX.TT','CAC40': '^FCHI.TT','DAX30': '^GDAXI.TT', 'SP500':'^GSPC.TT'},
                            ('Wiki','AlphaVantage'):{}}
        key_tuple = (source,target) # Tuple key to select Source-Target exceptions
        
        exception_dict = exceptions.get(key_tuple, {}) #FIX inform not properly loaded. Mirar longitud del dict.

        if name in exception_dict:
            return exception_dict.get(name, name) 
        
        name = name.replace(excedent_string,'')
        return prefix + name + suffix

In [ ]:
raw_ticker_list
parser_keys = {('market','IBEX35'):{'prefix':'','suffix':'.MC.TT','excedent_string':''},
               ('market','CAC40') :{'prefix':'','suffix':'.PA.TT','excedent_string':'Euronext: '},
               ('market','DAX30') :{'prefix':'','suffix':'.DE.TT', 'excedent_string':''},
               ('market','SP500') :{'prefix':'','suffix':'', 'excedent_string':''}}

In [ ]:
def parse_markets(raw_ticker, parser_keys): # Feeds tickers for Yahoo finance downloader based on market. Uses parse from BaseParser
        try:
            return parse(raw_ticker['ticker'], 
                     parser_keys[('market', raw_ticker['market'])]['prefix'],
                     parser_keys[('market', raw_ticker['market'])]['suffix'],
                     parser_keys[('market', raw_ticker['market'])]['excedent_string'])
        except:
            return raw_ticker['ticker']

In [ ]:
parser_keys[('market', raw_ticker_list[0]['market'])]

In [ ]:
parsed_tickers = parse_markets(raw_ticker_list[0], parser_keys)

In [ ]:
parsed_tickers

In [ ]:
parsed_tickers = parse_markets(raw_ticker_list[35], parser_keys)

In [ ]:
parsed_tickers

In [ ]:
market = 'IBEX35'
[append for append in parser_keys[('market', raw_ticker_list[0]['market'])]]
    

In [ ]:
parsed_tickers

In [ ]:
parsed_tickers = parse_markets(raw_ticker_list[35], parser_keys)

In [ ]:
parsed_tickers

In [ ]:
def parse_string_manipulation(name, prefix, suffix):
    print(name)
    name = name.replace('Euronext ','')
    return prefix + name + suffix

In [ ]:
parse_string_manipulation('Euronext IBEX','','.MC')

In [ ]:
baseParser = BaseParser('Wiki', 'YahooFinance')
name = 'CAC40_'
ticker = baseParser.parse(name, '', '.MC')
ticker
#feeder_ticker_list = my_parser_W2YF.parse_all(raw_ticker_list)

In [ ]:
dict_3 ={('Wiki','YahooFinance'):{'IBEX_1': '^IBEX35_1','CAC40': '^GSPC','DAX30': '^GDAXI'},
        ('Wiki','AlphaVantage'):{'IBEX_2': '^IBEX35_2','CAC40': '^GSPC','DAX30': '^GDAXI'},
        ('Wiki','Root'):{'IBEX_3': '^IBEX35_3','CAC40': '^GSPC','DAX30': '^GDAXI'}}

In [ ]:
target = 'Wiki'
source = 'YahooFinance'
#source = 'AlphaVantage'
#source = 'no'
name = 'IBEX_1'
print(name)

key_tuple = (target, source)
try:
    exception_dict = dict_3[key_tuple]
    #print(exception_dict)
except:
    exception_dict = {}
    print('!No exceptions loaded! Invalid source-target tuple key')
    print('Review possible data not loaded properly')

if name in exception_dict:
    name = exception_dict.get(name, name)

name

In [ ]:
parser_keys = {'IBEX35':{'maket':'IBEX35', 'prefix':'','suffix':'.MC'},
              'CAC40':{'maket':'CAC40','prefix':'','suffix':'.PA'},
              'DAX30':{'maket':'DAX30','prefix':'','suffix':'.DE'},
              'SP500':{'market':'SP500','prefix':'','suffix':''}}

parser_keys

In [ ]:
parser_keys['IBEX35']['prefix']

In [ ]:
raw_ticker_list[0]

In [ ]:
raw_ticker_list[0]['market'], raw_ticker_list[0]['ticker'] 

In [ ]:
enriched_ticker_list = []
raw_ticker_list[0]
raw_ticker_list[0]["Yahoo_API"] = raw_ticker_list[0]['ticker'] + ".MC"
raw_ticker_list[0]

In [ ]:
enriched_ticker_list.append(raw_ticker_list[0])
enriched_ticker_list

In [ ]:
def parse_tickers(raw_ticker_list):
    enriched_ticker_list = []
    for i, ticker_raw_metadata in enumerate(raw_ticker_list):
        if ticker_raw_metadata['market'] == 'IBEX35':
            #enriched['market'] = wiki_metadata['market']
            print("Enriched data for IBEX35")
        else:
            print("This ticker:", raw_ticker_list[0]['ticker'], "is not clasified and enriched" )
            
            
        #print(i, ticker_raw_metadata)

In [ ]:
#parse_tickers(raw_ticker_list)

In [ ]:
class BaseParser():
    
    def __init__(self, source = '', target =''):
        self.__source = ''
        self.__target = ''
        self.__exceptions = {
            'exception_1': 'fixed_1',
            'exception_2': 'fixed_2',
            'exception_3': 'fixed_3',
            
        }
    
    def parse(self, name, prefix, suffix):
        name = self.__exceptions.get(name, name) #Substitute ticker exception for requested ticker
        name = prefix + name + suffix
        #print(prefix, name, suffix)
        return name
            

class WikiToYahooFinanceParser(BaseParser):
    
    def __init__(self):
        super().__init__(source='Wiki', target='YahooFinance')
        
        self.__parser_keys = [{'market':'IBEX35', 'prefix':'', 'suffix':'.MC' }]
        
    def parse(self, raw_ticker):
        if raw_ticker['market'] == 'IBEX35':
            feed = super().parse(raw_ticker['ticker'], self.__parser_keys[0]['prefix'], self.__parser_keys[0]['suffix'])
            return feed
        else:
            return raw_ticker['ticker']
        
    def feeder_ticker(self,feed_ticker):
        feed_ticker['feeds'] = {'name':'CAMBIAR','ticker':self.parse(feed_ticker)}
        return feed_ticker
    
    def parse_all(self, raw_ticker_list):
        feed_ticker_list = []
        for raw_ticker in raw_ticker_list:
            feed_ticker_list.append(self.feeder_ticker(raw_ticker))
        return feed_ticker_list

In [ ]:
my_parser = BaseParser()
x = my_parser.parse('ELTON', '', '')

In [ ]:
print(x)

In [ ]:
exception_1 = {'ticker': 'exception_1',
 'company': 'Acciona',
 'sector': 'Construcción',
 'entry_date': '2015',
 'ISIN': 'ES0125220311',
 'market': 'IBEX35',
 'active_type': 'stock'}

In [ ]:
raw_ticker_list[0]

In [ ]:
my_parser_W2YF = WikiToYahooFinanceParser()
x = my_parser_W2YF.parse(raw_ticker_list[0])
print(x)
x = my_parser_W2YF.parse(exception_1)
print(x)


In [ ]:
my_list = raw_ticker_list[0]
print(my_list)
x = my_parser_W2YF.feeder_ticker(my_list)
x


In [ ]:
my_parser_W2YF = WikiToYahooFinanceParser()
feeder_ticker_list = my_parser_W2YF.parse_all(raw_ticker_list)


In [ ]:
print(feeder_ticker_list)

In [ ]:
def feeder_ticker(feed_ticker):
    feed_ticker['feeds'] = {'name':'AlphaVantage','ticker':my_parser_W2YF.parse(feed_ticker)}
    return feed_ticker

In [ ]:
feeder_ticker(raw_ticker_list[14])

In [ ]:
feed_ticker_list = []
for raw_ticker in raw_ticker_list[0:10]:
    feed_ticker_list.append(feeder_ticker(raw_ticker))

In [ ]:
feed_ticker_list